In [1]:
import torch as tc
import pandas as pd
from curve_utils import CurveUtils
device = tc.device("cuda" if tc.cuda.is_available() else "cpu")

In [2]:
their_semantics = [
    'Neck girth',
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Upper arm girth R',
    'Thigh girth R',
    'Height (m)',
    'Total crotch length',
    'Across back shoulder width',
]
our_semantic = [
    'neck_girth', # 5.3.2
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'upper_arm_girth', # 5.3.16
    'thigh_girth', # 5.3.20
    'stature', # 5.1.1
    'crotch_length', # 5.1.15
    'shoulder_width', # 5.4.3
]
curve_index = {
    'neck_girth':4, # 5.3.2
    'bust_chest_girth': 0, # 5.3.4
    'waist_girth': 1, # 5.3.10
    'hip_girth': 1, # 5.3.13
    'upper_arm_girth': 3, # 5.3.16
    'thigh_girth': 2, # 5.3.20
    'stature': 0,
    'crotch_length': 5
}
curves_names = list(curve_index.keys())

In [3]:
selected_subjects = dict()
selected_measures = dict()

for gender in ['female', 'male']:
    measures = pd.read_pickle(f'data/{gender}_measures.zip')
    measures.index = measures['Subject']
    measures = measures[measures['Measuring station'] == "MOVE4D"]
    temp_measures = measures[their_semantics].iloc[::2]
    
    mean_measures = measures._get_numeric_data().mean(axis=0)
    selected_subjects[gender] = 'IEEEP2_07' if gender == 'female' else 'IEEEP2_04'
    selected_measures[gender] = temp_measures.loc[selected_subjects[gender]]
    selected_subjects[gender] = temp_measures.index.get_loc(selected_subjects[gender])*2
    selected_measures[gender].index = our_semantic
    selected_measures[gender]['stature'] *= 1000

In [4]:
selected_subjects

{'female': 4, 'male': 2}

In [5]:
body_13 = CurveUtils.load_mesh("tests/female_4_body.obj")
template_13 = CurveUtils.load_template("tests/female_4_body.obj")[0]

# body_82 = CurveUtils.load_mesh("tests/female_4_body_82.obj")
# template_82 = CurveUtils.load_template("tests/female_4_body_82.obj")[0]

# body_330 = CurveUtils.load_mesh("tests/female_4_body_330.obj")
# template_330 = CurveUtils.load_template("tests/female_4_body_330.obj")[0]

# body_1322 = CurveUtils.load_mesh("tests/female_4_body_1322.obj")
# template_1322 = CurveUtils.load_template("tests/female_4_body_1322.obj")[0]

bodies = [
    (body_13.to(device) , template_13.to(device), "13k"),
    # (body_82.to(device) , template_82.to(device), "82k"),
    # (body_330.to(device), template_330.to(device), "330k"),
    # (body_1322.to(device), template_1322.to(device),"1M")
]

In [6]:
selected_measures

{'female': neck_girth           357.052
 bust_chest_girth    1071.830
 waist_girth          991.659
 hip_girth           1156.170
 upper_arm_girth      355.569
 thigh_girth          659.437
 stature             1620.000
 crotch_length        775.397
 shoulder_width       398.429
 Name: IEEEP2_07, dtype: float64,
 'male': neck_girth           404.586
 bust_chest_girth    1102.610
 waist_girth          964.188
 hip_girth           1081.650
 upper_arm_girth      330.421
 thigh_girth          647.553
 stature             1790.000
 crotch_length        784.841
 shoulder_width       469.971
 Name: IEEEP2_04, dtype: float64}

In [7]:
best_curves = tc.load(f'data/female_best_curves.zip')

for bodyset in bodies:

    print(f"calculating body: {bodyset[2]}")
    body = bodyset[0]
    template = bodyset[1]
    measures = selected_measures['female']
    result = CurveUtils.get_curves(body, selected_measures['female'], template, device)
    best_curves, data = CurveUtils.select_better(result, body, body, template, measures, device)
    print(data[:-1])

    # try:
    #     all_positions = []
    #     for curve in best_curves:
    #         position = CurveUtils.generate_positions(curve, body)
    #         position, measures = CurveUtils.ray_polygon_colision(body[template], position)
    #         all_positions.extend(position)
    #     all_positions = tc.row_stack(all_positions)
    #     CurveUtils.save_obj("teste.obj", all_positions.cpu().numpy())
    # except Exception as e:
    #     print(e)

calculating body: 13k
     original    measured  error(mm)
0   35.705200   35.704773   0.004272
1  107.182999  107.329384   1.463852
2   99.165901   99.164696   0.012054
3  115.617004  115.025162   5.918427
4   35.556900   35.690845   1.339455
5   65.943703   65.977676   0.339737
6  162.000000  162.018723   0.187225
